In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import os
import seaborn as sns


In [ ]:
CITY = 'DD'

In [ ]:

weather_filename_DD = f"../data/weather/df_Dresden_weather 2025-01-17_14-52-44.pkl"
weather_filename_FB = f"../data/weather/df_Freiburg_weather 2025-01-17_14-52-44.pkl"
weather_df_DD = pd.read_pickle(weather_filename_DD)
weather_df_FB = pd.read_pickle(weather_filename_FB)

In [ ]:
weather_df_DD

In [ ]:
datetime = "2025-03-19_10-47-56"
filename_DD = f"../data/nextbike/hourly_demand_supply_Dresden {datetime}.csv"
filename_FB = f"../data/nextbike/hourly_demand_supply_Freiburg {datetime}.csv"
df_DD = pd.read_csv(filename_DD, index_col=None, parse_dates=["datetime_hour"])
df_FB = pd.read_csv(filename_FB, index_col=None, parse_dates=["datetime_hour"])


help_dict = {'DD': df_DD, 'FB': df_FB}

df = help_dict[CITY]

In [ ]:
df.head()

# weekday, month

In [ ]:
plt.figure(figsize=(6, 5))
sns.reset_defaults()
sns.set_style("white")
sns.set_palette("husl")

sns.lineplot(data=avg_trips, x='weekday_rent', y='Count', hue='month_rent', style='month_rent')
plt.xlabel('Weekday')
plt.ylabel('Average number of trips per day')
if CITY=='DD':
    anchor=1.3
else:
    anchor=1.05

plt.legend(title="Month", bbox_to_anchor=(anchor, 1)) #plt.grid(True)
if CITY=='DD':
    plt.ylim(bottom=0, top=9000)
else:
    plt.ylim(bottom=0, top=4000)

plt.grid(True) 
#plt.savefig(f'/Users/v.sinichenko/Downloads/plots/seasonality_{CITY}_weekday.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
# flt = (df.weekday_rent=='Friday') & (df.month_rent=='January')
# len(df[flt])

In [ ]:
flt = (df.weekday_rent=='Friday') & (df.month_rent=='July')
len(df[flt].date_rent.unique())

In [ ]:
14404/4

In [ ]:
len(df)

In [ ]:
len(df.date_return.unique())

In [ ]:
len(df)/len(df.date_return.unique())

# hour, weekday

In [ ]:
grouping_cols = ['hour_rent', 'weekday_rent']

In [ ]:
ntrips = df.groupby(grouping_cols).size()

In [ ]:
ntrips

In [ ]:
nintervals = df.groupby(grouping_cols)['datetime_hour_rent'].nunique()

In [ ]:
nintervals

In [ ]:
avg_trips= ntrips/nintervals

In [ ]:
avg_trips.head(4)

In [ ]:
avg_trips = avg_trips.reset_index(name="Count")

In [ ]:
avg_trips

In [ ]:
# avg number of trips per hour
len(df)/len(df.datetime_hour_rent.unique())

In [ ]:
avg_trips.sort_values(by='Count', ascending=False)

In [ ]:
avg_trips["weekday_rent"] = pd.Categorical(avg_trips["weekday_rent"], categories=weekday_order, ordered=True)

In [ ]:
avg_trips.loc[avg_trips.weekday_rent=='Wednesday']

In [ ]:
plt.figure(figsize=(6, 5))
sns.reset_defaults()
sns.set_style("white")
sns.set_palette("husl")

sns.lineplot(data=avg_trips, x='hour_rent', y='Count', hue='weekday_rent', style='weekday_rent')
plt.ylabel('Average number of trips per hour')
plt.xlabel('Hour')
if CITY=='DD':
    anchor=1.05
else:
    anchor=1.05

plt.legend(title="Weekday", bbox_to_anchor=(anchor, 1)) #plt.grid(True)
if CITY=='DD':
    plt.ylim(bottom=0, top=2000)
else:
    plt.ylim(bottom=0, top=200)

plt.grid(True) 
plt.savefig(f'/Users/v.sinichenko/Downloads/plots/seasonality_{CITY}_hour_weekday.png', format='png', bbox_inches='tight')
plt.show()